In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import joblib
import pickle
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from string import digits
import string
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
from nltk import word_tokenize
from gensim.models import Word2Vec
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def default_vector():
        return np.zeros(100)
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    punctuation = string.punctuation
    return text.translate(str.maketrans('', '', punctuation))

def remove_digits(text):
    return text.translate(str.maketrans('', '', digits))

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def prepro(list):
    data = []
    for sentence in list:
        x = lemmatize_words(remove_digits(remove_punctuation(remove_html(remove_urls(sentence)))))
        data.append(x)
    return data

In [ ]:
def tf(list):
    with open("./weight/tfidf_vectorizer.pkl", "rb") as file:
        tfidfModel = pickle.load(file)
    tfidf_matrix = tfidfModel.transform(list).todense()
    load_model = joblib.load("./weight/finalized_model.sav")
    x = pd.DataFrame(tfidf_matrix)
    result = load_model.predict(x)
    sentiment = {'2': 'positive','1':'neutral','0':'negative'}
    result = [sentiment[str(senti)] for senti in result]
    return result

def we(list):
    modelw = Word2Vec.load("./weight/word_embedding.bin")
    vectors = [modelw.wv[word] if word in modelw.wv else default_vector() for word in list]
    load_model = joblib.load("./weight/finalized_model_WE.sav")
    result = load_model.predict(vectors)
    sentiment = {'2': 'positive','1':'neutral','0':'negative'}
    result = [sentiment[str(senti)] for senti in result]
    return result

def bow(list):
    with open("./weight/bow_vectorizer.pkl", "rb") as file:
        bowModel = pickle.load(file)
    bow_matrix = bowModel.transform(list).todense()
    load_model = joblib.load("./weight/finalized_model_bow.sav")
    x = pd.DataFrame(bow_matrix)
    result = load_model.predict(x)
    sentiment = {'2': 'positive','1':'neutral','0':'negative'}
    result = [sentiment[str(senti)] for senti in result]
    return result

In [ ]:
sentence = ["According to L+Ã±nnen Tehtaat 's CEO Matti Karppinen , the company aims to deliver fish products to its customers a day earlier than it currently does .",
      "Ahlstrom Corporation STOCK EXCHANGE ANNOUNCEMENT 7.2.2007 at 10.30 A total of 56,955 new shares of Ahlstrom Corporation have been subscribed with option rights under the company 's stock option programs I 2001 and II 2001 ."]
result = tf(prepro(sentence))
result

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


['positive', 'neutral']